In [3]:
# ! sudo mkdir -p /mnt/data

In [5]:
# cd /mnt/data

In [6]:
pwd

'/Users/olgabot/code/kmer-hashing/notebooks'

In [7]:
! aws s3 cp s3://olgabot-maca/fastqs.csv .

download: s3://olgabot-maca/fastqs.csv to ./fastqs.csv             


In [10]:
import pandas as pd

annotations = pd.read_csv('../../tabula-muris/00_data_ingest/18_global_annotation_csv/annotations_facs.csv')
annotations['cell'] = annotations['cell'].str.replace('.', '-')
annotations = annotations.rename(columns={'cell': 'cell_id'})
# annotations = annotations
annotations.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Neurog3>0_raw,Neurog3>0_scaled,cell_id,cell_ontology_class,cell_ontology_id,cluster.ids,free_annotation,mouse.id,mouse.sex,plate.barcode,...,subsetC,subsetC_cluster.ids,subsetD,subsetD_cluster.ids,subsetE,subsetE_cluster.ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2
0,NaN,NaN,A1-B000610-3_56_F-1-1,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_56_F,F,B000610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-7.182841,-10.895094
1,NaN,NaN,A1-B002764-3_38_F-1-1,bladder urothelial cell,CL:1001428,3,Luminal bladder epithelial cell,3_38_F,F,B002764,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,4.199059,-11.050240
2,NaN,NaN,A1-B002771-3_39_F-1-1,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_39_F,F,B002771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-11.995435,-7.325534
3,NaN,NaN,A1-D041914-3_8_M-1-1,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_8_M,M,D041914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-6.820022,-14.174246
4,NaN,NaN,A1-D042253-3_9_M-1-1,bladder cell,CL:1001319,2,Bladder mesenchymal cell,3_9_M,M,D042253,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-24.163538,-7.499349


In [11]:
pd.options.display.max_colwidth = 200

In [12]:
cols = ['cell_ontology_class', 'tissue', 'subtissue',  'free_annotation', 'cell_id']


def sanitize(x):
    try:
        return x.replace(" ", "_")
    except AttributeError:
        return str(x)

def join(keys, values):
    return '|'.join(f'{sanitize(key)}:{sanitize(value)}' 
                    for key, value in zip(keys, values))

sample_id_with_annotation = annotations.apply(lambda x: join(cols, x[cols]), axis=1)
sample_id_with_annotation.name = 'name'
sample_id_with_annotation.index = annotations['cell_id']
sample_id_with_annotation.head()

cell_id
A1-B000610-3_56_F-1-1                      cell_ontology_class:bladder_cell|tissue:Bladder|subtissue:nan|free_annotation:Bladder_mesenchymal_cell|cell_id:A1-B000610-3_56_F-1-1
A1-B002764-3_38_F-1-1    cell_ontology_class:bladder_urothelial_cell|tissue:Bladder|subtissue:nan|free_annotation:Luminal_bladder_epithelial_cell|cell_id:A1-B002764-3_38_F-1-1
A1-B002771-3_39_F-1-1                      cell_ontology_class:bladder_cell|tissue:Bladder|subtissue:nan|free_annotation:Bladder_mesenchymal_cell|cell_id:A1-B002771-3_39_F-1-1
A1-D041914-3_8_M-1-1                        cell_ontology_class:bladder_cell|tissue:Bladder|subtissue:nan|free_annotation:Bladder_mesenchymal_cell|cell_id:A1-D041914-3_8_M-1-1
A1-D042253-3_9_M-1-1                        cell_ontology_class:bladder_cell|tissue:Bladder|subtissue:nan|free_annotation:Bladder_mesenchymal_cell|cell_id:A1-D042253-3_9_M-1-1
Name: name, dtype: object

In [13]:
import pandas as pd

fastqs = pd.read_csv('fastqs.csv')
print(fastqs.shape)
fastqs = fastqs.rename(columns={'sample_id':'id', 'R1': 'read1', 'R2': 'read2'})

# Remove _S38, etc
fastqs['id'] = fastqs['id'].map(lambda x: '_'.join(x.split('_')[:-1]))

# Add annotation
fastqs = fastqs.set_index('id')
fastqs = fastqs.join(sample_id_with_annotation)

# Map ids to ids to fill NA
id_to_id = pd.Series(fastqs.index, index=fastqs.index)
fastqs['name'] = fastqs['name'].fillna(id_to_id)

fastqs.head()

(54912, 3)


,read1,read2,name
id,,,
A1-B000126-3_39_F-1-1,s3://czbiohub-maca/remux_data/170925_A00111_0066_AH3TKNDMXX/rawdata/A1-B000126-3_39_F-1-1_S289/A1-B000126-3_39_F-1-1_S289_R1_001.fastq.gz,s3://czbiohub-maca/remux_data/170925_A00111_0067_BH3M5YDMXX/rawdata/A1-B000126-3_39_F-1-1_S289/A1-B000126-3_39_F-1-1_S289_R2_001.fastq.gz,cell_ontology_class:epidermal_cell|tissue:Skin|subtissue:Telogen|free_annotation:Intermediate_IFE|cell_id:A1-B000126-3_39_F-1-1
A1-B000127-3_38_F-1-1,s3://czbiohub-maca/remux_data/170914_A00111_0058_AH3FYKDMXX/rawdata/A1-B000127-3_38_F-1-1_S1/A1-B000127-3_38_F-1-1_S1_R1_001.fastq.gz,s3://czbiohub-maca/remux_data/170914_A00111_0058_AH3FYKDMXX/rawdata/A1-B000127-3_38_F-1-1_S1/A1-B000127-3_38_F-1-1_S1_R2_001.fastq.gz,cell_ontology_class:endothelial_cell|tissue:Fat|subtissue:SCAT|free_annotation:nan|cell_id:A1-B000127-3_38_F-1-1
A1-B000167-3_56_F-1-1,s3://czbiohub-maca/remux_data/170925_A00111_0066_AH3TKNDMXX/rawdata/A1-B000167-3_56_F-1-1_S228/A1-B000167-3_56_F-1-1_S228_R1_001.fastq.gz,s3://czbiohub-maca/remux_data/170925_A00111_0066_AH3TKNDMXX/rawdata/A1-B000167-3_56_F-1-1_S228/A1-B000167-3_56_F-1-1_S228_R2_001.fastq.gz,cell_ontology_class:basal_cell|tissue:Mammary_Gland|subtissue:Mammary_Gland|free_annotation:nan|cell_id:A1-B000167-3_56_F-1-1
A1-B000168-3_57_F-1-1,s3://czbiohub-maca/remux_data/170925_A00111_0067_BH3M5YDMXX/rawdata/A1-B000168-3_57_F-1-1_S177/A1-B000168-3_57_F-1-1_S177_R1_001.fastq.gz,s3://czbiohub-maca/remux_data/170925_A00111_0067_BH3M5YDMXX/rawdata/A1-B000168-3_57_F-1-1_S177/A1-B000168-3_57_F-1-1_S177_R2_001.fastq.gz,A1-B000168-3_57_F-1-1
A1-B000412-3_56_F-1-1,s3://czbiohub-maca/remux_data/170914_A00111_0058_AH3FYKDMXX/rawdata/A1-B000412-3_56_F-1-1_S110/A1-B000412-3_56_F-1-1_S110_R1_001.fastq.gz,s3://czbiohub-maca/remux_data/170914_A00111_0058_AH3FYKDMXX/rawdata/A1-B000412-3_56_F-1-1_S110/A1-B000412-3_56_F-1-1_S110_R2_001.fastq.gz,cell_ontology_class:endothelial_cell|tissue:Heart|subtissue:RA|free_annotation:nan|cell_id:A1-B000412-3_56_F-1-1


In [14]:
ksizes = '21,27,33,51'
protein = 'true'
scaled = 1000
trim_low_abundance_kmers = 'true'

prefix = 's3://olgabot-maca/facs/sourmash_compute_all/'
fastqs['output'] = prefix + fastqs.index + '.sig'
# fastqs = fastqs.rename(columns={'sample_id': 'name', 'R1': 'read1', 'R2': 'read2'})

fastqs['ksizes'] = ksizes
fastqs['protein'] = protein
fastqs['scaled'] = scaled
fastqs['trim_low_abundance_kmers'] = trim_low_abundance_kmers

fastqs.head()

,read1,read2,name,output,ksizes,protein,scaled,trim_low_abundance_kmers
id,,,,,,,,
A1-B000126-3_39_F-1-1,s3://czbiohub-maca/remux_data/170925_A00111_0066_AH3TKNDMXX/rawdata/A1-B000126-3_39_F-1-1_S289/A1-B000126-3_39_F-1-1_S289_R1_001.fastq.gz,s3://czbiohub-maca/remux_data/170925_A00111_0067_BH3M5YDMXX/rawdata/A1-B000126-3_39_F-1-1_S289/A1-B000126-3_39_F-1-1_S289_R2_001.fastq.gz,cell_ontology_class:epidermal_cell|tissue:Skin|subtissue:Telogen|free_annotation:Intermediate_IFE|cell_id:A1-B000126-3_39_F-1-1,s3://olgabot-maca/facs/sourmash_compute_all/A1-B000126-3_39_F-1-1.sig,"21,27,33,51",true,1000,true
A1-B000127-3_38_F-1-1,s3://czbiohub-maca/remux_data/170914_A00111_0058_AH3FYKDMXX/rawdata/A1-B000127-3_38_F-1-1_S1/A1-B000127-3_38_F-1-1_S1_R1_001.fastq.gz,s3://czbiohub-maca/remux_data/170914_A00111_0058_AH3FYKDMXX/rawdata/A1-B000127-3_38_F-1-1_S1/A1-B000127-3_38_F-1-1_S1_R2_001.fastq.gz,cell_ontology_class:endothelial_cell|tissue:Fat|subtissue:SCAT|free_annotation:nan|cell_id:A1-B000127-3_38_F-1-1,s3://olgabot-maca/facs/sourmash_compute_all/A1-B000127-3_38_F-1-1.sig,"21,27,33,51",true,1000,true
A1-B000167-3_56_F-1-1,s3://czbiohub-maca/remux_data/170925_A00111_0066_AH3TKNDMXX/rawdata/A1-B000167-3_56_F-1-1_S228/A1-B000167-3_56_F-1-1_S228_R1_001.fastq.gz,s3://czbiohub-maca/remux_data/170925_A00111_0066_AH3TKNDMXX/rawdata/A1-B000167-3_56_F-1-1_S228/A1-B000167-3_56_F-1-1_S228_R2_001.fastq.gz,cell_ontology_class:basal_cell|tissue:Mammary_Gland|subtissue:Mammary_Gland|free_annotation:nan|cell_id:A1-B000167-3_56_F-1-1,s3://olgabot-maca/facs/sourmash_compute_all/A1-B000167-3_56_F-1-1.sig,"21,27,33,51",true,1000,true
A1-B000168-3_57_F-1-1,s3://czbiohub-maca/remux_data/170925_A00111_0067_BH3M5YDMXX/rawdata/A1-B000168-3_57_F-1-1_S177/A1-B000168-3_57_F-1-1_S177_R1_001.fastq.gz,s3://czbiohub-maca/remux_data/170925_A00111_0067_BH3M5YDMXX/rawdata/A1-B000168-3_57_F-1-1_S177/A1-B000168-3_57_F-1-1_S177_R2_001.fastq.gz,A1-B000168-3_57_F-1-1,s3://olgabot-maca/facs/sourmash_compute_all/A1-B000168-3_57_F-1-1.sig,"21,27,33,51",true,1000,true
A1-B000412-3_56_F-1-1,s3://czbiohub-maca/remux_data/170914_A00111_0058_AH3FYKDMXX/rawdata/A1-B000412-3_56_F-1-1_S110/A1-B000412-3_56_F-1-1_S110_R1_001.fastq.gz,s3://czbiohub-maca/remux_data/170914_A00111_0058_AH3FYKDMXX/rawdata/A1-B000412-3_56_F-1-1_S110/A1-B000412-3_56_F-1-1_S110_R2_001.fastq.gz,cell_ontology_class:endothelial_cell|tissue:Heart|subtissue:RA|free_annotation:nan|cell_id:A1-B000412-3_56_F-1-1,s3://olgabot-maca/facs/sourmash_compute_all/A1-B000412-3_56_F-1-1.sig,"21,27,33,51",true,1000,true


In [15]:
pwd

'/Users/olgabot/code/kmer-hashing/notebooks'

In [16]:
import json

folder = '../sourmash/maca/facs_compute_v7_all'

! mkdir -p $folder
fastqs.to_csv(f'{folder}/samples.csv')

config = 	{
		"program": "../../../reflow/sourmash_compute.rf",
		"runs_file": "samples.csv"
	}

with open(f'{folder}/config.json', 'w') as f:
    json.dump(config, f)
    
! head -n 3 $folder/samples.csv $folder/config.json

==> ../sourmash/maca/facs_compute_v7_all/samples.csv <==
id,read1,read2,name,output,ksizes,protein,scaled,trim_low_abundance_kmers
A1-B000126-3_39_F-1-1,s3://czbiohub-maca/remux_data/170925_A00111_0066_AH3TKNDMXX/rawdata/A1-B000126-3_39_F-1-1_S289/A1-B000126-3_39_F-1-1_S289_R1_001.fastq.gz,s3://czbiohub-maca/remux_data/170925_A00111_0067_BH3M5YDMXX/rawdata/A1-B000126-3_39_F-1-1_S289/A1-B000126-3_39_F-1-1_S289_R2_001.fastq.gz,cell_ontology_class:epidermal_cell|tissue:Skin|subtissue:Telogen|free_annotation:Intermediate_IFE|cell_id:A1-B000126-3_39_F-1-1,s3://olgabot-maca/facs/sourmash_compute_all/A1-B000126-3_39_F-1-1.sig,"21,27,33,51",true,1000,true
A1-B000127-3_38_F-1-1,s3://czbiohub-maca/remux_data/170914_A00111_0058_AH3FYKDMXX/rawdata/A1-B000127-3_38_F-1-1_S1/A1-B000127-3_38_F-1-1_S1_R1_001.fastq.gz,s3://czbiohub-maca/remux_data/170914_A00111_0058_AH3FYKDMXX/rawdata/A1-B000127-3_38_F-1-1_S1/A1-B000127-3_38_F-1-1_S1_R2_001.fastq.gz,cell_ontology_class:endothelial_cell|tissue:Fat|subtis